# 快速算出技術指標

In [213]:
import pandas as pd

In [214]:
df = pd.DataFrame({"code":[2330,2330,2330,1101,1101,1101],
                    "date":[20170808,20170809,20170810,20170808,20170809,20170810],
                    "open":[250,251,252,10,10,11],
                    "close":[250,250,254,11,12,13]})


In [215]:
df

,close,code,date,open
0,250,2330,20170808,250
1,250,2330,20170809,251
2,254,2330,20170810,252
3,11,1101,20170808,10
4,12,1101,20170809,10
5,13,1101,20170810,11


## 用dict建立data frame會按照column的字母順序排，所以自己先排

In [216]:
df=df[['code','date','open','close']]

In [217]:
df

,code,date,open,close
0,2330,20170808,250,250
1,2330,20170809,251,250
2,2330,20170810,252,254
3,1101,20170808,10,11
4,1101,20170809,10,12
5,1101,20170810,11,13


## 如果直接對raw data rollingmean，兩檔股票接續的地方算出的MA會有錯誤

In [218]:
MA2=df['close'].rolling(2).mean()
df.insert(loc=4,column="MA2",value=MA2)

In [219]:
df

,code,date,open,close,MA2
0,2330,20170808,250,250,NaN
1,2330,20170809,251,250,250.0
2,2330,20170810,252,254,252.0
3,1101,20170808,10,11,132.5
4,1101,20170809,10,12,11.5
5,1101,20170810,11,13,12.5


## 所以要依照股票代碼進行分群，計算MA

### 若group_keys=True，格式會不一樣，無法併進原df

In [220]:
df.groupby(['code'], group_keys=True).apply(lambda g:g.close.rolling(2).mean()).reindex()

code   
1101  3      NaN
      4     11.5
      5     12.5
2330  0      NaN
      1    250.0
      2    252.0
Name: close, dtype: float64

### 若group_keys=False，即可併進原df

In [221]:
df.groupby(['code'], group_keys=False).apply(lambda g:g.close.rolling(2).mean())

3      NaN
4     11.5
5     12.5
0      NaN
1    250.0
2    252.0
Name: close, dtype: float64

### 併入原df，會報錯但不影響結果

In [222]:
MA2=df.groupby(['code'], group_keys=False).apply(lambda g:g.close.rolling(2).mean())

In [225]:
df.loc[:,'MA2']=MA2

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [224]:
df

,code,date,open,close,MA2
0,2330,20170808,250,250,NaN
1,2330,20170809,251,250,250.0
2,2330,20170810,252,254,252.0
3,1101,20170808,10,11,NaN
4,1101,20170809,10,12,11.5
5,1101,20170810,11,13,12.5
